In [4]:
# Import Libraries
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set Spark Config
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("ShoppingTrendsComplexQueries") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Set Log Level
spark.sparkContext.setLogLevel("WARN")

# Load Data
try:
    # Attempt to load processed data first (Parquet)
    df = spark.read.parquet("/workspaces/CloudComputingITCS-6190-Project/data/processed/shopping")
    print("✓ Loaded processed data from parquet")
    print(f"✓ Loaded {df.count():,} records\n")
except:
    # Fallback to loading raw data (CSV) if parquet doesn't exist
    try:
        df = spark.read.csv("data/shopping.csv", header=True, inferSchema=True)
        print("✓ Loaded data from CSV")
        print(f"✓ Loaded {df.count():,} records\n")
    except Exception as e:
        print(f"Error loading data: {e}")
        # Create a dummy dataframe for demonstration if no file found (optional)
        # In a real notebook, you'd want to stop here.
        raise e

# Create Temporary View for SQL Queries
df.createOrReplaceTempView("shopping")

# Query 1: Top Categories Purchased per Age Group
print("\n" + "="*80)
print("QUERY 1: TOP CATEGORIES PURCHASED PER AGE GROUP")
print("="*80)
print("Business Insight: Understanding age-based product preferences")
print("-"*80 + "\n")

q1_result = spark.sql("""
    WITH age_category_stats AS (
        SELECT 
            CASE 
                WHEN Age < 25 THEN '18-24 (Gen Z)'
                WHEN Age BETWEEN 25 AND 34 THEN '25-34 (Millennials)'
                WHEN Age BETWEEN 35 AND 44 THEN '35-44 (Gen X)'
                WHEN Age BETWEEN 45 AND 54 THEN '45-54 (Gen X)'
                WHEN Age BETWEEN 55 AND 64 THEN '55-64 (Boomers)'
                ELSE '65+ (Seniors)'
            END as age_group,
            Category,
            COUNT(*) as purchase_count,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as total_revenue,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_purchase,
            ROUND(AVG(`Review Rating`), 2) as avg_rating
        FROM shopping
        GROUP BY age_group, Category
    ),
    ranked_categories AS (
        SELECT 
            age_group,
            Category,
            purchase_count,
            total_revenue,
            avg_purchase,
            avg_rating,
            ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY purchase_count DESC) as rank,
            ROUND(purchase_count * 100.0 / SUM(purchase_count) OVER (PARTITION BY age_group), 2) as percentage_of_age_group
        FROM age_category_stats
    )
    SELECT 
        age_group,
        Category,
        purchase_count,
        total_revenue,
        avg_purchase,
        avg_rating,
        percentage_of_age_group,
        rank as popularity_rank
    FROM ranked_categories
    WHERE rank <= 3
    ORDER BY age_group, rank
""")

q1_result.show(30, truncate=False)


# Query 2: Subscription Status Impact on Purchase Frequency
print("\n" + "="*80)
print("QUERY 2: SUBSCRIPTION STATUS IMPACT ON PURCHASE FREQUENCY")
print("="*80)
print("Business Insight: ROI of subscription program")
print("-"*80 + "\n")

q2_result = spark.sql("""
    WITH frequency_analysis AS (
        SELECT 
            `Subscription Status`,
            `Frequency of Purchases`,
            COUNT(*) as customer_count,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_purchase_amount,
            ROUND(AVG(`Previous Purchases`), 2) as avg_historical_purchases,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            ROUND(AVG(Age), 1) as avg_age
        FROM shopping
        GROUP BY `Subscription Status`, `Frequency of Purchases`
    ),
    subscription_summary AS (
        SELECT 
            `Subscription Status`,
            SUM(customer_count) as total_customers,
            ROUND(AVG(avg_purchase_amount), 2) as overall_avg_purchase,
            ROUND(AVG(avg_historical_purchases), 2) as overall_avg_history
        FROM frequency_analysis
        GROUP BY `Subscription Status`
    )
    SELECT 
        f.`Subscription Status`,
        f.`Frequency of Purchases`,
        f.customer_count,
        ROUND(f.customer_count * 100.0 / s.total_customers, 2) as pct_of_subscribers,
        f.avg_purchase_amount,
        f.avg_historical_purchases,
        f.avg_rating,
        f.avg_age,
        s.overall_avg_purchase as group_avg_purchase,
        s.overall_avg_history as group_avg_history
    FROM frequency_analysis f
    JOIN subscription_summary s ON f.`Subscription Status` = s.`Subscription Status`
    ORDER BY f.`Subscription Status`, f.customer_count DESC
""")

q2_result.show(20, truncate=False)

# Optional: Calculate subscription lift summary
print("\n📊 Subscription Program Analysis:")
q2_summary = spark.sql("""
    SELECT 
        `Subscription Status`,
        COUNT(*) as customers,
        ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_purchase,
        ROUND(AVG(`Previous Purchases`), 2) as avg_loyalty
    FROM shopping
    GROUP BY `Subscription Status`
    ORDER BY `Subscription Status` DESC
""")
q2_summary.show()


# Query 3: Discount & Promo Code Impact on Purchase Amounts
print("\n" + "="*80)
print("QUERY 3: DISCOUNT & PROMO CODE IMPACT ON PURCHASE AMOUNTS")
print("="*80)
print("Business Insight: Effectiveness of promotional strategies")
print("-"*80 + "\n")

q3_result = spark.sql("""
    WITH discount_promo_stats AS (
        SELECT 
            `Discount Applied`,
            `Promo Code Used`,
            COUNT(*) as transaction_count,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_purchase,
            ROUND(MIN(`Purchase Amount (USD)`), 2) as min_purchase,
            ROUND(MAX(`Purchase Amount (USD)`), 2) as max_purchase,
            ROUND(STDDEV(`Purchase Amount (USD)`), 2) as stddev_purchase,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as total_revenue,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            COUNT(CASE WHEN `Subscription Status` = 'Yes' THEN 1 END) as subscriber_count
        FROM shopping
        GROUP BY `Discount Applied`, `Promo Code Used`
    ),
    baseline AS (
        SELECT AVG(`Purchase Amount (USD)`) as baseline_avg
        FROM shopping
        WHERE `Discount Applied` = 'No' AND `Promo Code Used` = 'No'
    )
    SELECT 
        d.`Discount Applied`,
        d.`Promo Code Used`,
        d.transaction_count,
        d.avg_purchase,
        d.min_purchase,
        d.max_purchase,
        d.stddev_purchase,
        d.total_revenue,
        d.avg_rating,
        ROUND(d.subscriber_count * 100.0 / d.transaction_count, 2) as subscriber_rate,
        ROUND(d.transaction_count * 100.0 / (SELECT COUNT(*) FROM shopping), 2) as market_share_pct,
        ROUND((d.avg_purchase - b.baseline_avg) / b.baseline_avg * 100, 2) as vs_baseline_pct
    FROM discount_promo_stats d
    CROSS JOIN baseline b
    ORDER BY d.transaction_count DESC
""")

q3_result.show(truncate=False)


# Query 4: Payment Method Performance Analysis
print("\n" + "="*80)
print("QUERY 4: PAYMENT METHOD PERFORMANCE ANALYSIS")
print("="*80)
print("Business Insight: Payment preferences and spending correlation")
print("-"*80 + "\n")

q4_result = spark.sql("""
    WITH payment_stats AS (
        SELECT 
            `Payment Method`,
            COUNT(*) as transaction_count,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_purchase,
            ROUND(MEDIAN(`Purchase Amount (USD)`), 2) as median_purchase,
            ROUND(MIN(`Purchase Amount (USD)`), 2) as min_purchase,
            ROUND(MAX(`Purchase Amount (USD)`), 2) as max_purchase,
            ROUND(STDDEV(`Purchase Amount (USD)`), 2) as stddev,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as total_revenue,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            ROUND(AVG(Age), 1) as avg_customer_age,
            COUNT(CASE WHEN `Subscription Status` = 'Yes' THEN 1 END) as subscriber_count
        FROM shopping
        GROUP BY `Payment Method`
    ),
    ranked_payments AS (
        SELECT 
            *,
            RANK() OVER (ORDER BY avg_purchase DESC) as spending_rank,
            RANK() OVER (ORDER BY transaction_count DESC) as popularity_rank,
            ROUND(transaction_count * 100.0 / SUM(transaction_count) OVER (), 2) as market_share
        FROM payment_stats
    )
    SELECT 
        `Payment Method`,
        transaction_count,
        avg_purchase,
        median_purchase,
        min_purchase,
        max_purchase,
        stddev,
        total_revenue,
        avg_rating,
        avg_customer_age,
        ROUND(subscriber_count * 100.0 / transaction_count, 2) as subscriber_rate,
        market_share,
        spending_rank,
        popularity_rank
    FROM ranked_payments
    ORDER BY avg_purchase DESC
""")

q4_result.show(truncate=False)


# Query 5: Seasonal Category Performance Trends
print("\n" + "="*80)
print("QUERY 5: SEASONAL CATEGORY PERFORMANCE TRENDS")
print("="*80)
print("Business Insight: Seasonal demand patterns for inventory planning")
print("-"*80 + "\n")

q5_result = spark.sql("""
    WITH seasonal_performance AS (
        SELECT 
            Season,
            Category,
            COUNT(*) as sales_volume,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as revenue,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_price,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            COUNT(CASE WHEN `Discount Applied` = 'Yes' THEN 1 END) as discount_usage
        FROM shopping
        GROUP BY Season, Category
    ),
    season_totals AS (
        SELECT 
            Season,
            SUM(sales_volume) as season_total_sales
        FROM seasonal_performance
        GROUP BY Season
    ),
    ranked_seasonal AS (
        SELECT 
            sp.*,
            ROUND(sp.sales_volume * 100.0 / st.season_total_sales, 2) as pct_of_season,
            RANK() OVER (PARTITION BY sp.Season ORDER BY sp.revenue DESC) as revenue_rank,
            ROUND(sp.discount_usage * 100.0 / sp.sales_volume, 2) as discount_rate
        FROM seasonal_performance sp
        JOIN season_totals st ON sp.Season = st.Season
    )
    SELECT 
        Season,
        Category,
        sales_volume,
        revenue,
        avg_price,
        avg_rating,
        pct_of_season,
        discount_rate,
        revenue_rank
    FROM ranked_seasonal
    WHERE revenue_rank <= 3
    ORDER BY Season, revenue_rank
""")

q5_result.show(20, truncate=False)


# Query 6: Customer Lifetime Value Segmentation
print("\n" + "="*80)
print("QUERY 6: CUSTOMER LIFETIME VALUE SEGMENTATION")
print("="*80)
print("Business Insight: Identifying and profiling high-value customers")
print("-"*80 + "\n")

q6_result = spark.sql("""
    WITH customer_metrics AS (
        SELECT 
            `Customer ID`,
            Gender,
            Age,
            Location,
            COUNT(*) as total_transactions,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as lifetime_value,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_transaction,
            `Previous Purchases` as loyalty_score,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            MAX(`Subscription Status`) as is_subscriber
        FROM shopping
        GROUP BY `Customer ID`, Gender, Age, Location, `Previous Purchases`
    ),
    clv_scores AS (
        SELECT 
            *,
            NTILE(4) OVER (ORDER BY lifetime_value) as value_quartile,
            NTILE(4) OVER (ORDER BY total_transactions) as frequency_quartile,
            NTILE(4) OVER (ORDER BY loyalty_score DESC) as loyalty_quartile,
            ROUND((lifetime_value - AVG(lifetime_value) OVER ()) / STDDEV(lifetime_value) OVER (), 2) as value_zscore
        FROM customer_metrics
    ),
    segments AS (
        SELECT 
            *,
            CASE 
                WHEN value_quartile = 4 AND frequency_quartile = 4 THEN 'VIP Champions'
                WHEN value_quartile = 4 THEN 'High Spenders'
                WHEN frequency_quartile = 4 THEN 'Frequent Buyers'
                WHEN loyalty_quartile = 4 THEN 'Loyal Customers'
                WHEN value_quartile = 1 AND frequency_quartile = 1 THEN 'At Risk'
                ELSE 'Regular Customers'
            END as customer_segment
        FROM clv_scores
    )
    SELECT 
        customer_segment,
        COUNT(*) as customer_count,
        ROUND(AVG(lifetime_value), 2) as avg_lifetime_value,
        ROUND(AVG(total_transactions), 1) as avg_transactions,
        ROUND(AVG(avg_transaction), 2) as avg_transaction_size,
        ROUND(AVG(Age), 1) as avg_age,
        ROUND(AVG(loyalty_score), 1) as avg_loyalty_score,
        ROUND(AVG(avg_rating), 2) as avg_rating,
        ROUND(SUM(CASE WHEN is_subscriber = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as subscriber_rate,
        ROUND(SUM(lifetime_value), 2) as segment_total_value,
        ROUND(SUM(lifetime_value) * 100.0 / (SELECT SUM(lifetime_value) FROM segments), 2) as pct_of_total_revenue
    FROM segments
    GROUP BY customer_segment
    ORDER BY avg_lifetime_value DESC
""")

q6_result.show(truncate=False)


# Query 7: Shipping Preferences & Impact Analysis
print("\n" + "="*80)
print("QUERY 7: SHIPPING TYPE PREFERENCES & IMPACT ANALYSIS")
print("="*80)
print("Business Insight: Optimizing shipping options and costs")
print("-"*80 + "\n")

q7_result = spark.sql("""
    WITH shipping_analysis AS (
        SELECT 
            `Shipping Type`,
            Category,
            COUNT(*) as order_count,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_order_value,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as total_revenue,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            ROUND(AVG(Age), 1) as avg_customer_age,
            COUNT(CASE WHEN `Subscription Status` = 'Yes' THEN 1 END) as subscriber_orders,
            COUNT(CASE WHEN `Discount Applied` = 'Yes' THEN 1 END) as discounted_orders
        FROM shopping
        GROUP BY `Shipping Type`, Category
    ),
    shipping_totals AS (
        SELECT 
            `Shipping Type`,
            SUM(order_count) as total_orders,
            ROUND(AVG(avg_order_value), 2) as overall_avg_order
        FROM shipping_analysis
        GROUP BY `Shipping Type`
    )
    SELECT 
        sa.`Shipping Type`,
        sa.Category,
        sa.order_count,
        sa.avg_order_value,
        sa.total_revenue,
        sa.avg_rating,
        sa.avg_customer_age,
        ROUND(sa.subscriber_orders * 100.0 / sa.order_count, 2) as subscriber_rate,
        ROUND(sa.discounted_orders * 100.0 / sa.order_count, 2) as discount_rate,
        ROUND(sa.order_count * 100.0 / st.total_orders, 2) as pct_of_shipping_type,
        st.overall_avg_order as shipping_type_avg
    FROM shipping_analysis sa
    JOIN shipping_totals st ON sa.`Shipping Type` = st.`Shipping Type`
    ORDER BY sa.`Shipping Type`, sa.total_revenue DESC
""")

q7_result.show(20, truncate=False)


# Query 8: Geographic Market Performance Analysis
print("\n" + "="*80)
print("QUERY 8: GEOGRAPHIC MARKET PERFORMANCE ANALYSIS")
print("="*80)
print("Business Insight: Regional opportunities and market penetration")
print("-"*80 + "\n")

q8_result = spark.sql("""
    WITH location_metrics AS (
        SELECT 
            Location,
            COUNT(DISTINCT `Customer ID`) as unique_customers,
            COUNT(*) as total_transactions,
            ROUND(SUM(`Purchase Amount (USD)`), 2) as total_revenue,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_transaction,
            ROUND(AVG(Age), 1) as avg_customer_age,
            ROUND(AVG(`Review Rating`), 2) as avg_rating,
            COUNT(CASE WHEN `Subscription Status` = 'Yes' THEN 1 END) as subscribers,
            COUNT(CASE WHEN Gender = 'Male' THEN 1 END) as male_customers,
            COUNT(CASE WHEN Gender = 'Female' THEN 1 END) as female_customers,
            ROUND(AVG(`Previous Purchases`), 1) as avg_loyalty
        FROM shopping
        GROUP BY Location
    ),
    top_category_per_location AS (
        SELECT 
            Location,
            Category as top_category,
            category_sales
        FROM (
            SELECT 
                Location,
                Category,
                COUNT(*) as category_sales,
                ROW_NUMBER() OVER (PARTITION BY Location ORDER BY COUNT(*) DESC) as rn
            FROM shopping
            GROUP BY Location, Category
        )
        WHERE rn = 1
    )
    SELECT 
        lm.Location,
        lm.unique_customers,
        lm.total_transactions,
        ROUND(lm.total_transactions * 1.0 / lm.unique_customers, 2) as transactions_per_customer,
        lm.total_revenue,
        lm.avg_transaction,
        lm.avg_customer_age,
        lm.avg_rating,
        ROUND(lm.subscribers * 100.0 / lm.total_transactions, 2) as subscription_rate,
        ROUND(lm.male_customers * 100.0 / lm.total_transactions, 2) as male_pct,
        ROUND(lm.female_customers * 100.0 / lm.total_transactions, 2) as female_pct,
        lm.avg_loyalty,
        tc.top_category,
        ROUND(lm.total_revenue * 100.0 / (SELECT SUM(total_revenue) FROM location_metrics), 2) as revenue_market_share
    FROM location_metrics lm
    JOIN top_category_per_location tc ON lm.Location = tc.Location
    ORDER BY lm.total_revenue DESC
    LIMIT 20
""")

q8_result.show(20, truncate=False)


# Query 9: Product Affinity & Market Basket Analysis
print("\n" + "="*80)
print("QUERY 9: PRODUCT AFFINITY & MARKET BASKET ANALYSIS")
print("="*80)
print("Business Insight: Product combinations for cross-selling strategies")
print("-"*80 + "\n")

q9_result = spark.sql("""
    WITH customer_categories AS (
        SELECT DISTINCT
            `Customer ID`,
            Category,
            COUNT(*) as category_purchases,
            ROUND(AVG(`Purchase Amount (USD)`), 2) as avg_spend_in_category
        FROM shopping
        GROUP BY `Customer ID`, Category
    ),
    category_pairs AS (
        SELECT 
            a.Category as category_A,
            b.Category as category_B,
            COUNT(DISTINCT a.`Customer ID`) as customers_buying_both,
            ROUND(AVG(a.avg_spend_in_category + b.avg_spend_in_category), 2) as avg_combined_spend,
            ROUND(AVG(a.category_purchases + b.category_purchases), 1) as avg_combined_purchases
        FROM customer_categories a
        JOIN customer_categories b 
            ON a.`Customer ID` = b.`Customer ID`
            AND a.Category < b.Category
        GROUP BY a.Category, b.Category
    ),
    category_totals AS (
        SELECT 
            Category,
            COUNT(DISTINCT `Customer ID`) as total_customers
        FROM shopping
        GROUP BY Category
    ),
    affinity_metrics AS (
        SELECT 
            cp.*,
            cta.total_customers as category_A_customers,
            ctb.total_customers as category_B_customers,
            (SELECT COUNT(DISTINCT `Customer ID`) FROM shopping) as total_unique_customers,
            ROUND(cp.customers_buying_both * 1.0 / cta.total_customers, 3) as support_A,
            ROUND(cp.customers_buying_both * 1.0 / ctb.total_customers, 3) as support_B,
            ROUND(
                (cp.customers_buying_both * 1.0 / (SELECT COUNT(DISTINCT `Customer ID`) FROM shopping)) / 
                ((cta.total_customers * 1.0 / (SELECT COUNT(DISTINCT `Customer ID`) FROM shopping)) * (ctb.total_customers * 1.0 / (SELECT COUNT(DISTINCT `Customer ID`) FROM shopping))),
                2
            ) as lift_score
        FROM category_pairs cp
        JOIN category_totals cta ON cp.category_A = cta.Category
        JOIN category_totals ctb ON cp.category_B = ctb.Category
    )
    SELECT 
        category_A,
        category_B,
        customers_buying_both,
        avg_combined_spend,
        avg_combined_purchases,
        ROUND(customers_buying_both * 100.0 / total_unique_customers, 2) as market_penetration_pct,
        support_A as confidence_A_to_B,
        support_B as confidence_B_to_A,
        lift_score,
        CASE 
            WHEN lift_score > 1.5 THEN 'Strong Affinity'
            WHEN lift_score > 1.2 THEN 'Moderate Affinity'
            WHEN lift_score > 1.0 THEN 'Weak Affinity'
            ELSE 'Negative Affinity'
        END as affinity_strength
    FROM affinity_metrics
    WHERE customers_buying_both >= 5
    ORDER BY lift_score DESC
    LIMIT 20
""")

q9_result.show(20, truncate=False)

# Stop Spark Session
spark.stop()

✓ Loaded processed data from parquet
✓ Loaded 27,533 records


QUERY 1: TOP CATEGORIES PURCHASED PER AGE GROUP
Business Insight: Understanding age-based product preferences
--------------------------------------------------------------------------------



+-------------------+-----------+--------------+-------------+------------+----------+-----------------------+---------------+
|age_group          |Category   |purchase_count|total_revenue|avg_purchase|avg_rating|percentage_of_age_group|popularity_rank|
+-------------------+-----------+--------------+-------------+------------+----------+-----------------------+---------------+
|18-24 (Gen Z)      |Clothing   |1522          |91526        |60.14       |3.76      |44.70                  |1              |
|18-24 (Gen Z)      |Accessories|1077          |65460        |60.78       |3.78      |31.63                  |2              |
|18-24 (Gen Z)      |Footwear   |514           |31034        |60.38       |3.74      |15.10                  |3              |
|25-34 (Millennials)|Clothing   |2354          |142226       |60.42       |3.75      |43.91                  |1              |
|25-34 (Millennials)|Accessories|1729          |102490       |59.28       |3.74      |32.25                  |2

25/12/05 21:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 2

+--------------+-----------------+------------+---------------+------------+------------+------+-------------+----------+----------------+---------------+------------+-------------+---------------+
|Payment Method|transaction_count|avg_purchase|median_purchase|min_purchase|max_purchase|stddev|total_revenue|avg_rating|avg_customer_age|subscriber_rate|market_share|spending_rank|popularity_rank|
+--------------+-----------------+------------+---------------+------------+------------+------+-------------+----------+----------------+---------------+------------+-------------+---------------+
|Credit Card   |4733             |60.17       |60.0           |20          |100         |23.67 |284765       |3.75      |44.4            |26.66          |17.19       |1            |3              |
|PayPal        |4795             |60.05       |60.0           |20          |100         |24.01 |287934       |3.74      |44.1            |26.65          |17.42       |2            |2              |
|Debit Car

25/12/05 21:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 21:03:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/05 2

+-----------------+--------------+------------------+----------------+--------------------+-------+-----------------+----------+---------------+-------------------+--------------------+
|customer_segment |customer_count|avg_lifetime_value|avg_transactions|avg_transaction_size|avg_age|avg_loyalty_score|avg_rating|subscriber_rate|segment_total_value|pct_of_total_revenue|
+-----------------+--------------+------------------+----------------+--------------------+-------+-----------------+----------+---------------+-------------------+--------------------+
|VIP Champions    |6172          |102.49            |1.3             |85.95               |44.0   |25.3             |3.76      |26.46          |632559             |38.42               |
|High Spenders    |251           |84.26             |1.0             |84.26               |44.8   |25.4             |3.75      |25.90          |21149              |1.28                |
|Frequent Buyers  |251           |68.67             |2.0             |